In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras import regularizers


from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.utils import load_img

from tqdm.notebook import tqdm
from keras.models import Model
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

# Age Classification Convolutional Neural Network

## Overview
We will build two neural networks to classify age, gender.

### Age Classification
The second neural network will classify age into the following bins:

- <span style="color:blue">Age 1-2</span>
- <span style="color:green">Age 3-9</span>
- <span style="color:orange">Age 10-20</span>
- <span style="color:red">Age 21-27</span>
- <span style="color:">Age 28-45</span>
- <span style="color:purple">Age 46-65</span>
- <span style="color:yellow">Age +66</span>




# Data Import

## Dataset Selection
We will use an augmented version of a dataset called <span style="color:blue">"UTKFace"</span>. This dataset comprises over <span style="color:green">100,000 images</span> of individuals ranging from <span style="color:orange">1 to 100 years</span> of age. In addition we will use another dataset for testing which compromises of around  <span style="color:green">40,000 images</span> of individuals ranging from <span style="color:orange">1 to 100 years</span> of age.

### File Information Extraction
To kickstart our data visualization process, we'll import the filenames into a dataframe.Extracting the details encapsulated within the filenames, including <span style="color:blue">name</span>, <span style="color:green">age</span>.



In [ ]:
BASE_DIR_train ="/kaggle/input/augmented-age-dataset/augmented_age_dataset/content/combined_faces_train_augmented/"

BASE_DIR_test ="/kaggle/input/age-test-dataset/content/combined_faces/"



In [ ]:
age_classes=['1-2','3-12','13-19','20-29','30-45','46-65','+66']



In [ ]:
image_paths_train = []
age_labels_train = []

for filename in tqdm(os.listdir(BASE_DIR_train)):
    image_path = os.path.join(BASE_DIR_train, filename)
    temp = filename.split('_')
    age = int(temp[0])
    image_paths_train.append(image_path)
    age_labels_train.append(age)


In [ ]:
image_paths_test = []
age_labels_test = []

for filename in tqdm(os.listdir(BASE_DIR_test)):
    image_path = os.path.join(BASE_DIR_test, filename)
    temp = filename.split('_')
    age = int(temp[0])
    image_paths_test.append(image_path)
    age_labels_test.append(age)


In [ ]:
df_age_train = pd.DataFrame()
df_age_train['image'], df_age_train['age'] = image_paths_train, age_labels_train
df_age_train.head()

In [ ]:
df_age_test = pd.DataFrame()
df_age_test['image'], df_age_test['age'] = image_paths_test, age_labels_test
df_age_test.head()

# Age Binning
We will transform the ages into <span style="color:blue">bins</span> for better classification.

## Data Validation

### Results
The results were successful. We managed to import our data and store it into our variables.
riables.


In [ ]:
img = Image.open(df_age_train['image'][0])
plt.axis('off')
plt.imshow(img);

In [ ]:
img=cv2.imread(df_age_train['image'][0])
img.shape

In [ ]:
def age_bins(age):

    if 1 <= age <= 2:
        return '1-2'
    elif 3 <= age <= 12:
        return '3-12'
    elif 13 <= age <= 19:
        return '13-19'
    elif 20 <= age <= 29:
        return '20-29'
    elif 30 <= age <= 45:
        return '30-45'
    elif 46 <= age <= 65:
        return '46-65'
    else:
        return '+66'

In [ ]:
def age_bins_encoded(age):

    if 1 <= age <= 2:
        return 0
    elif 3 <= age <= 12:
        return 1
    elif 13 <= age <= 19:
        return 2
    elif 20 <= age <= 29:
        return 3
    elif 30 <= age <= 45:
        return 4
    elif 46 <= age <= 65:
        return 5
    else:
        return 6


In [ ]:
df_age_train['age_bins']=df_age_train['age'].apply(age_bins)
df_age_test['age_bins']=df_age_test['age'].apply(age_bins)

In [ ]:
df_age_train['age_enc']=df_age_train['age'].apply(age_bins_encoded)
df_age_test['age_enc']=df_age_test['age'].apply(age_bins_encoded)

In [ ]:
df_age_train

In [ ]:
df_age_test

# Data Visualization






### Age Distribution (Training Dataset)

- **28-45:** <span style="color:green">65,490 instances</span>
- **21-27:** <span style="color:blue">46,840 instances</span>
- **46-65:** <span style="color:red">39,190 instances</span>
- **1-2:** <span style="color:orange">22,340 instances</span>
- **10-20:** <span style="color:purple">21,950 instances</span>
- **3-9:** <span style="color:green">19,710 instances</span>
- **+66:** <span style="color:pink">18,880 instances</span>

The dataset showcases a diverse age distribution. The largest representation is observed in the age range of 28-45, followed by 21-27 and 46-65. Notably, there's a substantial presence of individuals aged 1-2 and 10-20, indicating a significant representation of younger age groups. The dataset also includes a considerable number of instances in the 3-9 age range. Additionally, there's a noticeable representation of individuals aged 66 and above.

Overall, the dataset demonstrates diversity in age representation, encompassing a wide range of age groups. However, the predominance of younger age groups may impact model performance and generalization, particularly in age-related predictions or analyses.


### Age Distribution

The dataset displays a balanced distribution between age groups, with a slightly higher representation of adults and young adults compared to other age groups.

### Age Distribution (Test Dataset)

- **28-45:** <span style="color:green">9,356 instances</span>
- **21-27:** <span style="color:blue">6,691 instances</span>
- **46-65:** <span style="color:red">5,598 instances</span>
- **1-2:** <span style="color:orange">3,192 instances</span>
- **10-20:** <span style="color:purple">3,136 instances</span>
- **3-9:** <span style="color:green">2,816 instances</span>
- **+66:** <span style="color:pink">2,697 instances</span>

The test dataset exhibits a similar age distribution pattern to the training dataset, with the highest representation in the 28-45 and 21-27 age groups. However, the test dataset contains substantially fewer instances across all age bins compared to the training dataset. This difference in data volume and distribution between the training and test datasets may impact model evaluation and generalization.

The skew towards younger age groups, particularly 28-45 and 21-27, in both datasets may influence model performance and generalization. It's crucial to consider this distribution during model evaluation to ensure robust performance across all age categories.



In [ ]:
df_age_train['age_bins'].value_counts()

In [ ]:
df_age_test['age_bins'].value_counts()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

sns.histplot(x=df_age_train['age_bins'], hue='age_bins', data=df_age_train)
plt.title('Age Distribution Histogram (Training Dataset)')
plt.xlabel('Age Categories')
plt.show(),

In [ ]:
sns.histplot(x=df_age_test['age_bins'], hue='age_bins', data=df_age_test)
plt.title('Age Distribution Histogram (Test Dataset)')
plt.xlabel('Age Categories')
plt.show(),

## Data Preparation and Dataframe Creation

### Loading and Preprocessing
- **train_image_list**: <span style="color:blue">Training</span> image file paths.
- **train_labels_list**: <span style="color:blue">Training</span> age labels.
- **test_image_list**: <span style="color:orange">Test</span> image file paths.
- **test_labels_list**: <span style="color:orange">Test</span> age labels.
- **train_image_list_tf**: <span style="color:blue">Training</span> image file paths (TensorFlow constant).
- **train_labels_list_tf**: <span style="color:blue">Training</span> age labels (TensorFlow constant).
- **test_image_list_tf**: <span style="color:orange">Test</span> image file paths (TensorFlow constant).
- **test_labels_list_tf**: <span style="color:orange">Test</span> age labels (TensorFlow constant).
- **num_classes**: <span style="color:green">Number</span> of age categories (7).

### Parsing Function
- **_parse_function(filename, label)**: Parses images and labels.
    - Reads image files.
    - Decodes JPEG images.
    - Converts labels to one-hot encoded vectors.


In [ ]:
train_image_list = list(df_age_train['image'])
train_labels_list = list(df_age_train['age_enc'])

test_image_list = list(df_age_test['image'])
test_labels_list = list(df_age_test['age_enc'])

In [ ]:
train_image_list_tf = tf.constant(train_image_list)
train_labels_list_tf = tf.constant(train_labels_list)

test_image_list_tf = tf.constant(test_image_list)
test_labels_list_tf = tf.constant(test_labels_list)

In [ ]:
num_classes = 7
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)  
    
    image_resized = tf.image.resize(image_decoded, [200,200])
    
    image_decoded = tf.image.random_flip_left_right(image_decoded) 
    
    image_decoded = tf.image.random_brightness(image_decoded, max_delta=0.2)
    
    image_decoded = tf.image.random_contrast(image_decoded, lower=0.5, upper=1.5)
    
    image_decoded= tf.cast(image_decoded, tf.float32)

    
    image_normalized = image_decoded / 255.0

    
    label = tf.one_hot(label, num_classes)

    return image_normalized, label

In [ ]:

train_image_dataset = tf.data.Dataset.from_tensor_slices((train_image_list_tf, train_labels_list_tf))
train_image_dataset = train_image_dataset.map(_parse_function)
train_image_dataset = train_image_dataset.repeat(2)

train_image_dataset = train_image_dataset.batch(312) 


test_image_dataset = tf.data.Dataset.from_tensor_slices((test_image_list_tf, test_labels_list_tf))
test_image_dataset = test_image_dataset.map(_parse_function)
test_image_dataset = test_image_dataset.repeat(2)

test_image_dataset = test_image_dataset.batch(312) 

In [ ]:
train_image_dataset

# Age Classification Model

## Model Architecture

The age classification model is a Convolutional Neural Network (CNN) with several convolutional layers followed by max-pooling layers for feature extraction. Batch normalization is applied after each convolutional layer for normalization and faster training. Dropout regularization is also used to prevent overfitting.

After the convolutional layers, a Global Average Pooling layer is added to reduce the spatial dimensions of the feature maps. Two dense layers with ReLU activation functions are then applied for further feature processing.

The output layer employs a <span style="color:orange">softmax activation function</span> to predict the probability distribution across the age categories.

Model Summary:
- Input Shape: (Image Height, Image Width, 3)
- Convolutional Layers: <span style="color:blue">64, 128, 256, and 512 filters</span> with a kernel size of (3, 3)
- Max Pooling Layers: Applied after each convolutional layer
- Dropout Layer: Applied with a dropout rate of <span style="color:green">0.3</span> after the last convolutional layer
- Global Average Pooling Layer
- Dense Layers: <span style="color:blue">256 and 128 units</span> with ReLU activation function
- Batch Normalization: Applied after dense layers
- Output Layer: Dense layer with <span style="color:blue">7 units</span> and softmax activation function for age classification

The model architecture effectively extracts features from input images and classifies them into different age categories.


## Compilation
The model is compiled using the <span style="color:blue">Adam</span> with a learning rate of <span style="color:green">0.001</span>. <span style="color:orange">Categorical cross-entropy loss</span> is utilized as the loss function, which is suitable for multi-class classification tasks.

## Model Training
Training is conducted using a batch size of <span style="color:green">312</span> over <span style="color:green">18</span> epochs. <span style="color:orange">Early stopping</span> with a patience of <span style="color:green">5</span> is employed to prevent overfitting, and the best model weights are restored. The training and validation data are evaluated for accuracy metrics.

## Results

The age classification model was trained over 18 epochs, yielding the following insights:

- **Initial Epoch:**
  - The model commenced with an accuracy of <span style="color:orange">51.07%</span> and a loss of <span style="color:orange">1.1936</span>, indicating modest performance.

- **Progressive Improvement:**
  - Subsequent epochs demonstrated consistent enhancements in both accuracy and loss metrics, reflecting improved model learning.

- **Final Epoch:**
  - By the 30th epoch, the model achieved a notable accuracy of <span style="color:green">96.28%</span> with a reduced loss of <span style="color:green">0.1024</span>.

- **Validation Performance:**
  - The validation metrics exhibited fluctuations throughout training, with accuracy ranging from <span style="color:green">43.32%</span> to <span style="color:green">81.57%</span>, and loss varying between <span style="color:green">0.7069</span> and <span style="color:green">2.2826</span>.

These observations suggest progressive improvement in model performance over the training epochs, culminating in satisfactory accuracy and loss metrics by the final epoch. However, fluctuations in validation performance indicate potential challenges in generalization to unseen data.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


model = tf.keras.models.Sequential()


model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(200,200,1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))


model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())


model.add(layers.Dense(7, activation='softmax'))  


model.summary()

In [ ]:


early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)


callbacks_list=[early_stopping_callback]

model.compile(optimizer="adam", loss=['categorical_crossentropy'], metrics=['accuracy'])


                

In [ ]:

history=model.fit(
    train_image_dataset,
    batch_size=312,    
    epochs=30,
    callbacks=callbacks_list,
    validation_data=test_image_dataset,
    verbose=2,
    shuffle=False


)


# Results Visualization

In [ ]:
y_predict = model.predict(test_image_dataset)


In [ ]:
def argmax_fun(array):
   array_1=[]
   for i in array:
       if i == array.max():
            i=1
       else:
            i=0
       array_1.append(i)
   return array_1

In [ ]:
y_predict_argmax= y_predict.copy()
for i in range(len(y_predict)):
    y_predict_argmax[i]= argmax_fun(y_predict[i])

In [ ]:
y_predict_argmax

In [ ]:
y_val_list=[]
for i in y_age_val:
    y_val_list.append(i.argmax())

In [ ]:
cm_age = tf.math.confusion_matrix(labels=list(df_age_test['age_enc']), predictions=y_predict_argmax_list, num_classes=4)
conf_matrix_result = cm_age.numpy()
cm_age= sns.heatmap(cm_age, annot=True, fmt=".1f")
tick_positions = np.arange(4) + 0.5
cm_age.set_yticks(tick_positions, age_classes)

cm_age.set_xticks(tick_positions, age_classes)



## Interpretation of Confusion Matrix

The confusion matrix provides a visual representation of the model's performance in classifying age categories. Here's an interpretation based on the results:

- **<span style="color:green">High Correct Classifications (True Positives)</span>**: The diagonal elements of the confusion matrix indicate the number of instances correctly classified for each age category. For example, the model correctly classified 7013 instances as belonging to the first age category (20-27 years old).

- **<span style="color:red">Misclassifications (False Positives)</span>**: Off-diagonal elements represent instances that were incorrectly classified as belonging to a different age category. For instance, there were 68 instances classified as belonging to the second age category (20-27 years old) but actually belonged to the first category.

- **<span style="color:blue">Results</span>**: By examining the distribution of correct and incorrect classifications across age categories, we can identify any notable patterns or trends. For example, people aged between 20 and 27 have a high chance of being labeled as a person between the age of 28 and 45. 



In [ ]:
conf_matrix_result


In [ ]:
epochs = range(len(history.history["loss"]))
plt.plot(epochs, history.history["loss"], 'blue', label = "loss")
plt.plot(epochs, history.history["val_loss"], 'red', label ="val_loss")
plt.title("Total Loss and Total Validation Loss")
plt.legend()

In [ ]:
epochs = range(len(history.history["loss"]))
plt.plot(epochs, history.history["accuracy"], 'blue', label = "accuracy")
plt.plot(epochs, history.history["val_accuracy"], 'red', label ="val_accuracy")
plt.title("Total Accuracy and Total Validation Accuracy")
plt.legend()

In [ ]:
model.save('age_model_aug.h5')
